<a href="https://colab.research.google.com/github/ArunD2018/arund.gethub.io/blob/master/Bert_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  U

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
from Bio import Entrez
import pandas as pd
from operator import itemgetter
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from tqdm.notebook import tqdm

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('MELISA database-2').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# Convert to a DataFrame and render.

In [ ]:
def format_str(x):
  if x!="":
    x = x.replace('\xa0',' ').lower().rstrip("s").split(" ")
    format_x = []
    for i in x:
      format_i = wnl.lemmatize(i.replace(",",""))
      formati = ","+format_i if i.find(",")==0 else format_i+"," if i.find(",")>0 else format_i
      format_x.append(formati)
    return " ".join(format_x)
  else:
    return None

In [ ]:
b = map(itemgetter(3), rows[2:])
wnl = WordNetLemmatizer()
mesh= set(map(format_str,list(filter(bool, b))))

In [ ]:
def search(query):
    Entrez.email = 'jeevanamaddineni@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='1000',
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

In [ ]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'arunindiran@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [ ]:
abstract_data = []
for i in tqdm(mesh):
  results = search(i)
  id_list = results['IdList']
  if len(id_list)!=0:
    papers = fetch_details(id_list)
    for j, paper in enumerate(papers['PubmedArticle']):
      if 'Abstract' in list(paper['MedlineCitation']['Article'].keys()):
        abstract_data.append([i, paper['MedlineCitation']['Article']['ArticleTitle']," ".join(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])])


  0%|          | 0/75 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame(abstract_data).to_csv("/drive/MyDrive/Topic modelling/abstract_data1.csv",sep="\t")

In [ ]:
abstact_df = pd.read_csv("/drive/MyDrive/Topic modelling/abstract_data1.csv",sep="\t",index_col=[0])

In [ ]:
abstact_df['2'] = abstact_df['2'].replace("\\u","",regex=False)
abstact_df['2'] = abstact_df['2'].replace("(\d+(?:\.\d+)?)"," ",regex=True)
abstact_df['2'] = abstact_df['2'].replace("\([^)]*\)"," ",regex=True)
abstact_df['2'] = abstact_df['2'].replace("\[[^)]*\]"," ",regex=True)
abstact_df['2'] = abstact_df['2'].replace("\<[^)]*\>"," ",regex=True)
abstact_df= abstact_df.dropna(subset=['2'])
abstact_df['2'] = abstact_df['2'].dropna().apply(lambda x : " ".join(list(filter(str.isalpha,x.split(" ")))))

In [ ]:
from bertopic import BERTopic
bert_model = BERTopic(verbose=True, calculate_probabilities=True, min_topic_size=150,nr_topics=150)
topics, probabilities = bert_model.fit_transform(abstact_df['2'].to_list())

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2146 [00:00<?, ?it/s]

2021-10-19 14:31:26,957 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-10-19 14:35:36,068 - BERTopic - Reduced dimensionality with UMAP
2021-10-19 14:36:09,897 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-10-19 14:36:28,890 - BERTopic - Reduced number of topics from 42 to 42


In [ ]:
topics_per_class = bert_model.topics_per_class(abstact_df['2'].to_list(),topics , classes=abstact_df['0'])

73it [01:14,  1.01s/it]


In [ ]:
bert_model.visualize_topics_per_class(topics_per_class)

In [ ]:
bert_model.get_topic_info()

,Topic,Count,Name
0,-1,47312,-1_family_sexual_health_minority
1,31,3718,31_hiv_sex_risk_sexual
2,36,2441,36_pregnancy_gestation_fetal_maternal
3,35,1954,35_phenotypic_genetic_species_genes
4,32,1533,32_fathers_mothers_maternal_father
5,2,1140,2_meals_meal_obesity_eating
6,3,661,3_minority_minorities_disparities_heterosexual
7,13,532,13_family_families_childcare_functioning
8,19,512,19_fertility_longevity_family_children
9,1,449,1_discrimination_health_gsm_support


In [ ]:
bert_model.get_topic(36)

[('pregnancy', 0.03689110142937195),
 ('gestation', 0.02147202569423581),
 ('fetal', 0.01630245644809532),
 ('maternal', 0.014918269436809947),
 ('pregnant', 0.011934014430237878),
 ('progesterone', 0.011158066989772134),
 ('uterine', 0.010830060080103714),
 ('gestational', 0.01014305398101811),
 ('cows', 0.009019583889028534),
 ('placental', 0.008736599339131032)]

In [ ]:
bert_model.save("/drive/MyDrive/Topic modelling/My_topic_model")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
from bertopic import BERTopic
bert_model = BERTopic.load("/drive/MyDrive/Topic modelling/My_topic_model")

In [ ]:
bert_model.get_topic_info(36)

,Topic,Count,Name
2,36,2441,36_pregnancy_gestation_fetal_maternal


In [ ]:
representative_docs

{0: ['Medical education must ensure that future physicians receive comprehensive and culturally competent training to prepare them to care for SGM Based on our this training should include specific content to help medical students identify potential biases toward SGM as well as strategies to acknowledge and confront these',
  'The Coronavirus pandemic has deepened gender and racial diversity problems in Mentorship shows women and other groups where the ladders to success and helps them avoid the a revised leaky pipeline we identify tangible strategies that will improve gender including increasing active mentorship by male',
  'Sexual and gender minority young adults have higher smoking prevalence than their Less is known about differences in smoking characteristics within the SGM Participants were SGM young adult smokers age enrolled in a clinical trial of the Put It Out a Facebook smoking cessation intervention for SGM young Analyses tested differences between sexual orientation and g

In [ ]:
bert_model.visualize_topics()

NameError: name 'bert_model' is not defined

In [ ]:
bert_model.visualize_hierarchy()

In [ ]:
bert_model.visualize_barchart(top_n_topics=17,height=1500)

In [ ]:
bert_model.visualize_heatmap()

In [ ]:
bert_model.visualize_term_rank()